# REST API Using Django

https://medium.com/@mathur.danduprolu/a-beginners-guide-to-building-rest-apis-with-python-and-django-rest-framework-ac9153d9ab7a

# What is an API?
> **API:** Application Programming Interfaces allow different softwares to communicate with each other.

One of the most common types of APIs is the REST API 
## What is a REST API?
> **REST API:** Representational State Transfer API; an architectural style for creating networked applications that rely on stateless, client-server communication

- **Client-server:** there are two sides:
    - Client (your app, browser, etc.) asks for information.
    - Server gives back the information.

- **Stateless:** each request is independent. The server doesn’t “remember” what you asked before. Every time the client talks to the server, it must include all the info the server needs to answer.

It allows clients to interact with servers by making requests over the HTTP protocol.  
Requests include (CRUD):
- **GET (READ):** Retrieve data from the server
- **POST (CREATE):** Send data to the server to create a new resource
- **PUT (UPDATE):** Update an existing resource on the server
- **DELETE:** Remove a resource from the server


REST APIs are widely used because they are simple, scalable, and use the same basic HTTP operations that web browsers use to fetch and send data.

## Django REST Framework (DRF)
> **Django:** Django provides a structured and efficient framework for building robust and scalable web applications using Python.

> **DRF:** A powerful toolkit to build Web APIs in Django

It offers an easy-to-use interface to build REST APIs and comes with many features such as authentication, serialization, and viewsets that make developing APIs easier.

# Create REST API Using Django:

## Step 1: Install Django and Django REST Framework:

In [4]:
# In cmd
# Install virtualenv if not already installed
pip install virtualenv

# Create a virtual environment
virtualenv venv

# Activate the virtual environment
venv\Scripts\activate  # For Windows

#Now install django and DFR
pip install django djangorestframework

# Create requirements file optional:
pip freeze > requirements.txt

SyntaxError: invalid syntax (2880399114.py, line 3)

## Step 2: Create a New Django Project:

A project is the overall container (the whole website or service) and inside it, you create smaller apps, each focused on one area of functionality.  
There are a few base modules created when running the following.

In [ ]:
django-admin startproject restapi_project
cd restapi_project

## Step 3: Create a New Django App:
To keep things moduler (clean and organised) create a new app within my project that will handle the REST API.

In [ ]:
python manage.py startapp api

## Step 4: Set up Django REST Framework:
In the restapi_project/settings.py module, add 'rest_framework' to the INSTALLED_APPS section to tell Django to enable the DRF so I can start building APIs with it.

In [ ]:
INSTALLED_APPS = [
    # other installed apps,
    'rest_framework',
    'api',  # newly created app
]

# Step 5: Define your Model
For demonstration, we will create a simple model for managing a list of tasks.

In [5]:
# in the api/models.py, define the 'Task' model:

from django.db import models

class Task(models.Model):
    """
    Task model for managing to-do style tasks.

    Fields:
        title (CharField): A short name or title of the task.
        description (TextField): A longer text giving details about the task.
        completed (BooleanField): Marks whether the task is done (True) or not (False).

    Methods:
        __str__: Returns the task's title when the object is printed or displayed.
    """
    # Short text field for the task's title (e.g., "Buy groceries")
    title = models.CharField(max_length=255)

    # Larger text field for details about the task (e.g., "Buy milk, bread, and eggs")
    description = models.TextField()

    # Boolean value that indicates if the task is completed or not
    completed = models.BooleanField(default=False)

    def __str__(self):
        """
        Returns the title of the task as its string representation.
        Helpful when viewing objects in the Django admin or shell.
        """
        return self.title


ModuleNotFoundError: No module named 'django'

### Create necessery databases.  
- We need a database to store, organise and retrieve data in a structured manner.
- For example, we may want to store all the tasks, completed or not.
- Django uses SQLite by default.

**makemigrations**
- Looks at each model (eg Task) and decides what changes need to be made in the database
- It creates a migration file describing those changes
- eg you see below the migration is creating a new table for the Task model with necessery fields. 

In [7]:
python manage.py makemigrations 
## migration file produced:
class Migration(migrations.Migration):

    initial = True

    dependencies = [
    ]

    operations = [
        migrations.CreateModel(
            name='Task',
            fields=[
                ('id', models.BigAutoField(auto_created=True, primary_key=True, serialize=False, verbose_name='ID')),
                ('title', models.CharField(max_length=255)),
                ('description', models.TextField()),
                ('completed', models.BooleanField(default=False)),
            ],
        ),
    ]


SyntaxError: invalid syntax (405299674.py, line 1)

**migrate:**
- Apply the migration files to the database
- This creates or updates tables in the database to match the models. (Task table will be created)

In [ ]:
python manage.py migrate
# db.sqlite3 is created

## Step 6: Create a Serializer:
Serializers in DRF are responsible for converting complex data types like Django models (e.g. my Task) into JSON (format used to send data over APIs), XML  or other content types and vice versa. 


In api/serializers.py, create a serializer for the Task model:

In [ ]:
from rest_framework import serializers
from .models import Task

class TaskSerializer(serializers.ModelSerializer):
    """
    Serializer for the Task model.

    Purpose:
        - Converts Task model instances into JSON (for API responses).
        - Converts JSON input into Task model instances (for saving to the database).

    Meta:
        model (Task): The model this serializer is based on.
        fields ('__all__'): Includes all fields of the Task model 
                            (title, description, completed).
    """

    class Meta:
        # Connect this serializer to the Task model
        model = Task

        # Include all fields from the Task model in the serializer
        # (title, description, completed will be automatically included)
        fields = '__all__'


### Nested Meta class:
Above, the nested Meta class is a **static configuration** for the class itself, so it doesnt change per object/instance (why we dont use __init__).  
We could in theory implement this using a class decorator, but we dont:
- **Python Convention:** Django uses nested Meta classes for configuration
- **Inheritance Friendly:** DRF inspects the serializer class, looks for Meta and automatically generates fields and validation before an instance has been created. The nested class makes this easier.


The actual serialization logic lives in ModelSerializer and BaseSerializer.  
When you use a `ModelSerializer`, here's what happens internally:

1. `ModelSerializer` looks at `Meta.model` (`Task`) and `Meta.fields` (`__all__`).

2. It dynamically creates serializer fields for `title`, `description`, and `completed`.

3. When you access `serializer.data`, it calls the base `to_representation` method, 
   which converts the `Task` instance into a Python dictionary.

In [ ]:
## example usage:
from api.models import Task
from api.serializers import TaskSerializer

# Get the first task from the database
task = Task.objects.first()

# Create a serializer instance
serializer = TaskSerializer(task)

# Convert the Task instance to a Python dictionary (ready for JSON)
print(serializer.data)


In [8]:
### Example Output
```json
{
  "id": 1,
  "title": "Buy groceries",
  "description": "Milk, eggs, bread",
  "completed": false
}


SyntaxError: invalid syntax (2204506537.py, line 2)

## Step 7: Create Views and URLs

### Django REST Framework Views

**Views** are the part of Django that **handles incoming requests** from clients.  

- When someone visits a URL or calls your API, Django passes the request to a **view**.  
- The view decides what to do and what data to send back.  

---

### API Views in DRF

In Django REST Framework (DRF), there are two main ways to make API views:

1. **APIView**  
   - Lets you define each method manually:  
     - `get()` → read data  
     - `post()` → create data  
     - `put()` / `patch()` → update data  
     - `delete()` → delete data
2. **ViewSet**  
   - A shortcut that automatically provides CRUD operations


In api/views.py, create the following views:

In [ ]:
from rest_framework.response import Response
from rest_framework.decorators import api_view
from rest_framework import status
from .models import Task
from .serializers import TaskSerializer

@api_view(['GET', 'POST']) #decorator (adds extra logic to function) that allows this function to handle only the specified HTTP methods.
def task_list(request):
    """
    Handle requests for the list of tasks.

    GET:
        - Retrieve all Task objects from the database.
        - Serialize them into JSON and return in the response.

    POST:
        - Accept JSON data to create a new Task.
        - Validate the data using TaskSerializer.
        - Save to the database if valid, else return errors.
    """
    if request.method == 'GET':
        # Get all tasks from the database
        tasks = Task.objects.all()

        # Serialize multiple objects with many=True
        serializer = TaskSerializer(tasks, many=True)

        # Return serialized data as JSON
        return Response(serializer.data)

    elif request.method == 'POST':
        # Deserialize incoming JSON data
        serializer = TaskSerializer(data=request.data)

        # Validate the data
        if serializer.is_valid():
            # Save the new Task to the database
            serializer.save()
            # Return created task with 201 status code
            return Response(serializer.data, status=status.HTTP_201_CREATED)

        # Return validation errors with 400 status code
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)


@api_view(['GET', 'PUT', 'DELETE'])
def task_detail(request, pk):
    """
    Handle requests for a single task by its primary key (id).

    GET:
        - Retrieve a single Task and return it as JSON.

    PUT:
        - Update a Task with incoming JSON data.
        - Validate and save changes if valid.

    DELETE:
        - Remove the Task from the database.
    """
    try:
        # Try to get the task by primary key
        task = Task.objects.get(pk=pk)
    except Task.DoesNotExist:
        # Return 404 if the task does not exist
        return Response(status=status.HTTP_404_NOT_FOUND)

    if request.method == 'GET':
        # Serialize the task object
        serializer = TaskSerializer(task)
        # Return serialized data as JSON
        return Response(serializer.data)

    elif request.method == 'PUT':
        # Deserialize incoming data and update the existing task
        serializer = TaskSerializer(task, data=request.data)
        if serializer.is_valid():
            # Save updates to the database
            serializer.save()
            return Response(serializer.data)
        # Return validation errors
        return Response(serializer.errors, status=status.HTTP_400_BAD_REQUEST)

    elif request.method == 'DELETE':
        # Delete the task from the database
        task.delete()
        # Return 204 No Content to indicate successful deletion
        return Response(status=status.HTTP_204_NO_CONTENT)


# Step 8: Define URLs in Django REST Framework

Create URLs to map the views. 

**URLs** are like **addresses** for your API endpoints.  
When someone visits a URL, Django checks this list to see **which view should handle the request**.

---

In api/urls.py, define the API routes:

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('tasks/', views.task_list),
    path('tasks/<int:pk>/', views.task_detail),
]

**`tasks/`** → points to the `task_list` view  
- Handles **GET** (list all tasks)  
- Handles **POST** (create a new task)  

**`tasks/<int:pk>/`** → points to the `task_detail` view  
- `<int:pk>` means it expects a **task ID number**  
- Handles **GET** (get one task), **PUT** (update task), **DELETE** (delete task)  

---

### Analogy

Think of URLs like **street addresses**:  

- `tasks/` → the main street where all tasks live  
- `tasks/3/` → the house of task with ID 3  

When a request comes in, Django checks **which “house” (view)** to send it to.


Now include these URLs in your project’s main urls.py:


In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),   # /admin/ → Django admin site
    path('api/', include('api.urls')), # /api/ → include all URLs defined in api/urls.py
]

## Step 9: Testing the API

Run the Django development server:

python manage.py runserver

By default, it runs at http://127.0.0.1:8000/
So your API endpoints will start with http://127.0.0.1:8000/api/

### Create a Postman Collection
This will group all your task endpoints together.  
- In Postman, click Collections → New Collection. 
- Name it something like Task API.
- Add each request below to this collection.

# CRUD Operations for Tasks API

### 1. Get All Tasks
- **Method:** GET  
- **URL:** `http://127.0.0.1:8000/api/tasks/`  

Click **Send** → you should see a list of tasks (empty list `[]` if none).

---

### 2. Create a New Task
- **Method:** POST  
- **URL:** `http://127.0.0.1:8000/api/tasks/`  

Go to **Body → raw → JSON**  

Example JSON:


In [1]:
{
  "title": "My First Task",
  "description": "This is a sample task",
  "completed": false
}

NameError: name 'false' is not defined

In [1]:
### 3. Get a Single Task
- **Method:** GET  
- **URL:** `http://127.0.0.1:8000/api/tasks/1/`  
  *(replace `1` with the actual task ID returned in step 2)*  

Click **Send** → should return that specific task.

---

### 4. Update a Task
- **Method:** PUT  
- **URL:** `http://127.0.0.1:8000/api/tasks/1/`  

Go to **Body → raw → JSON**  

Example JSON:

SyntaxError: invalid character '→' (U+2192) (698596579.py, line 6)

In [ ]:
{
  "title": "Updated Task Title",
  "description": "Updated description",
  "completed": true
}

### 5. Delete a Task
- **Method:** DELETE  
- **URL:** `http://127.0.0.1:8000/api/tasks/1/`  

Click **Send** → should return `204 No Content` (task deleted).
